# Calculation of transit windows using new routines.
- This set of routines is trying to improve on the original version of the transit planner. Right now it is in state of development, with the core features being written. Several bugs are likely still present, though they should be more target-specific. Please report any bugs on the GitHub page.

## Main changes are:
- Code is written in way that it can be called as a library, so we no longer need to re-run/ save the target_list values from `target_selection.py`.
- Currently, only NASA exoplanet archive is considered. The table is loaded automatically through TAP service, which is then stored locally as pickle file.
- Transit windows now have an optimal observation window information, which gives sufficient baseline for both transmission spectroscopy and Rossiter-McLaughlin effect observation.
- Transit windows are ranked based on quality. This is the main reason (together with p2 API system) this code was done.
- For transmission spectroscopy, a simulated dataset can be done (for now only for Na doublet line, though it will be expanded upon).
- Code should be properly documented, and generally easier to expand on.

## Known bugs:
- The observation window selection fails in some low quality transits (very visibly). 
    - This issue should be fixed
- The quality of windows violating Moon constraint should be worse.


## To be done:
- Optimalization:
    - The code currently runs slightly slower than the old planner. Furthermore, at high number of targets it might break due to memory issues.
        - The first issue is not overly significant on my laptop (this can still be run relativelly fast). Please tell me if you need it to be faster. The reasoning is that currently most arrays are calculated even for very bad transit windows (e.g., non-visible targets), so a preselection would solve this issue. There is also an additional "overhead" due to loading of NASA archive through API on the first run or when updating the table.
        - It should also be possible to do multiprocessing here.
        - Second issue is likely solved. I am keeping this info here for logkeeping.
- Re-adding lower priority features:
    - Parameters like SNR from RMR for transit, TSM etc.
    - Possibly: Other tables of exoplanets. However, many of these are not as precise as NASA archive is.
- Simulation of RM contamination on to the transmission spectrum.
- Adding custom ephemeris:
    - Currently, there are two ways to add custom ephemeris.
        1. Through `Exoplanets.add_custom_ephemeris()` function to load a single ephemeris set.
        2. Through `Exoplanets.load_csv_ephemeris()` function to load csv file.
    - More options will be considered based on user-cases.
- Reimplement the NASA_exo.py using the rats.parameters CompositeTable class (personal pipeline, will be added separately).
    - This shouldn't change the usage of the library, but it will provide additional functionalities (plots)


### Example on the TOI-132b.
1. First, lets import relevant libraries

In [ ]:
import os
os.chdir('/media/chamaeleontis/Observatory_main/Code/observations_transits/OT/OT')
from NASA_exo import NASA_Exoplanets_archive, logger, load_csv_ephemeris
from utilities import logger_default
import calculate_transit_windows as ctw
import astropy.time as astime
import astropy.units as u
# import simulate_transmission_spectrum as sts

2. Load the NASA exoplanet archive
- By using the `force_reload` keyword, you force the TAP service to be reloaded. Regardless of this keyword, on first time it will always be loaded anyway, since no saved data will be found.
- `use_PS` keyword allows to disable the `Planetary Systems` table, and only uses the composite table. This is generally not prefered, and is set to `False` by default. If set to true, the table is still loaded to fix several parameter issues in composite table (e.g., parameteres without errorbars, which would later break the code), but afterwards the table is lost. 
- `full_PS` keyword forces usage of the full `Planetary Systems` table from NASA archive, which is *huge*. It will take easily an hour to load, so by default it is disabled. If disabled, only the Ephemeris related parameters are loaded ($P$, $T_{14}$ and $T_{c}$)

In [ ]:
Exoplanets = NASA_Exoplanets_archive(
    force_reload = True,
    # use_PS = False,
    full_PS = False
    )

3. Filter out target sample based on criteria:
- I tried to make this part as close as to `target_selection.py` filtering. The main difference is the keywords are not changed to the more natural ones, but used from the NASA exoplanet archive table [link_to_definitions_here](https://exoplanetarchive.ipac.caltech.edu/docs/API_PS_columns.html).
- The pandas table is saved in `Exoplanets` class as `nasa_table_composite` (`Planetary systems Composite` table) and `nasa_table_all` (`Planetary systems`). Therefore all filters are done on `Exoplanets.nasa_table_composite` and `Exoplanets.nasa_table_all`, both of which are pandas DataFrames.
- For the filters to be used, use: `Exoplanets.filter_table_composite(condition)` or `Exoplanets.filter_table_all(condition)`. This returns a DataFrame that can be checked for output, but otherwise is unused. The used table is stored in `Exoplanets.filtered_table_composite` and `Exoplanet.filtered_table_all` keys. The filter is automatically binded between the tables, so filters for composite table are applied to full table and vice-versa.

In [ ]:
#%% Condition for filtering table
condition = (Exoplanets.nasa_table_composite['pl_name'].isin(
    ['TOI-132 b',]
    ))
# The filtered_table argument is not used later, but is returned for checking the outputs.
filtered_table = Exoplanets.filter_table_composite(condition)

4. Decide on period during which to calculate the transit windows:
- A simple function is defined for the ESO semester numbering, using the `ctw.get_dates_for_ESO_semester()` function.
- The dates are saved in `astime.Time` object, which is astropy Time package allowing for conversion between dates. This is used in the code, so simple 'yyyymmdd' format won't work. However, if necessary a convenient function can be written as well. The commented line shows how you can custom define a specific date.


In [ ]:
#%% Get current large program dates
P113 = ctw.get_dates_for_ESO_semester(P_number= 113,
                                        large_program=False)

# P112_115 = astime.Time(['2023-12-16 12:00:00.000', '2024-04-01 12:00:00.000'],scale = 'utc')

5. Calculate transit windows:
- This step calculates *all* transit windows location, regardless of their validity and location.
- `database` is the Exoplanets table from which to look for parameters of planets that have been previously selected.
- The `semester_date` is the time where we look for the windows.
- `save_directory` creates main folder where to save (eventually) the plots.

In [ ]:
Transit_windows = ctw.Transit_windows(
    database = Exoplanets,
    semester_date = P113,
    save_directory = '/media/chamaeleontis/Observatory_main/ESO_scheduling/CTW_ATREIDES_P113'
    )

6. Plot transit windows:
- Plots transit windows, provided location.
- `Location` must be defined in the `observatories.py` `Observatories` Enum. If your observatory is not defined, you can define it there. `Astropy` will very likely have it in a list, but otherwise it can be also defined manually. Check the documentation for the class on how to do it.

In [ ]:
Transit_windows.plot_transit_windows(Location = 'Paranal')

Optional:
- Simulation of transmission spectrum for given transit.
    - This takes as input the stellar and planetary parameters.
    - It creates a stellar synthetic spectrum using PHOENIX
    - Creates an estimated dataset (currently only ESPRESSO) assuming 900s of exposures.
    - Injects a sodium doublet signal based on empirical trend in the planet spectra
    - Calculates the synthetic transmission spectrum
    - To do: 
        - Fit the signal and compare it with the injected signal
        - Check the strength of the signal

